In [1]:
import os
import spacy
import tensorflow as tf
from utils.processor import DataProcessor

2023-07-19 16:50:43.763610: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-19 16:50:43.877800: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-19 16:50:43.878526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 16:50:44.481529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
if not os.path.exists('data/dataset/'):
    processor = DataProcessor(nlp)
    processor.load_data('data')
    processor.process_data()
    train, val, test = processor.test_train_split()

else:
    processor = DataProcessor(nlp, tf.data.Dataset.load('data/dataset/'))
    train, val, test = processor.test_train_split()

2023-07-19 16:50:51.064664: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-19 16:50:51.064994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_pr

In [8]:
processor.dataset

<_LoadDataset element_spec=(TensorSpec(shape=(300,), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [4]:
train = train.batch(64).cache().prefetch(1000)
val = val.batch(64).cache().prefetch(1000)
test = test.batch(64).cache().prefetch(1000)

In [5]:
train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(300,), dtype=tf.float32),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(len(processor.label_encoding.keys()))
])

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.AdamW(),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),
               tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)

In [7]:
model.fit(
    train,
    epochs=50,
    validation_data=val
)

Epoch 1/50


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 3188739365850266019


ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.